<a href="https://colab.research.google.com/github/AnthusAI/LangChain-Examples/blob/main/notebooks/AI_API_Multiplexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example 1: AI API Multiplexer

This example shows how you can connect to many different AI APIs using the same code by going through LangChain.

You can run this code if you have an OpenAI API key.  If you have other API keys then you can run some of the demos that use multiple AI APIs from the same code.

## Setup code

The first code cell installs the LangChain modules and retrieves AI API credentials.  Please set up Colab secrets for any of these:

* `OPENAI_API_KEY`

In [ ]:
# Install necessary libraries
!pip install langchain langchain-community langchain-core langchain-openai

# Get the OpenAI API key
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

# Set it as an environment variable
import os
os.environ["OPENAI_API_KEY"] = api_key

# Show more about what's happening as it happens.
from langchain.globals import set_debug
set_debug(True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 750.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141

## Example 1A: One simple LangChain request

This

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name="gpt-4o")
print(model.invoke("Is the Sagrada Família located in Barcelona?").content)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Is the Sagrada Família located in Barcelona?"
  ]
}
[llm/end] [llm:ChatOpenAI] [1.70s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the Sagrada Família is located in Barcelona, Spain. It's a renowned and unfinished basilica designed by the famous Catalan architect Antoni Gaudí. The construction of the Sagrada Família began in 1882 and continues to this day, making it one of the longest-running architectural projects in the world. The basilica is a UNESCO World Heritage Site and one of Barcelona's most iconic landmarks.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          

## Example 1A: Two Sequential Requests, Two Different Models

This example

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# First LLM: Use gpt-4o for the CoT prompt to get a detailed answer
cot_prompt_template = """
Question: {question}
Please provide a detailed explanation before answering whether the statement is true or false.
"""

cot_prompt = PromptTemplate(
    template=cot_prompt_template,
    input_variables=["question"])

# Create the first chain using OpenAI's gpt-4o model with a CoT prompt
chain_1 = ChatOpenAI(temperature=0.7, model_name="gpt-4o")

# Example question
question = "Is the Eiffel Tower located in Paris?"

# Invoke the first chain
cot_response = chain_1.invoke(cot_prompt.format(question=question))
detailed_answer = cot_response.content
print("Detailed answer:", detailed_answer)

# Second LLM: Use gpt-4o-mini to categorize the detailed answer as 'Yes' or 'No'
yes_no_prompt_template = """
Based on the following answer, please categorize it strictly as either "Yes" or "No":
Answer: "{detailed_answer}"
"""

yes_no_prompt = PromptTemplate(
    template=yes_no_prompt_template,
    input_variables=["detailed_answer"])

# Create the second chain using OpenAI's gpt-4o-mini model for yes/no clarification
chain_2 = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# Invoke the second chain
yes_no_response = chain_2.invoke(yes_no_prompt.format(detailed_answer=detailed_answer))
yes_no_answer = yes_no_response.content.strip()
print("Yes/No Answer:", yes_no_answer)